In [174]:
# Initial imports
import numpy as np
import pandas as pd
import hvplot.pandas
from collections import Counter
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from seaborn import set_style
set_style("whitegrid")

In [175]:
# import Final_project_NYC_BIKE_LANE_cleaned.csv data to match accident site 
file_path = "Resources/Segment2/Final_project_NYC_Bike_All_ML.csv"
ML = pd.read_csv(file_path)
print(ML.shape)
ML

(9193, 28)


,COLLISION_ID,DATES,TIME,DAY_OF_WEEK,MONTH,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN,BIKE_LANE
0,4271563,"January 2, 2020",13:00:00,Thursday,January,MANHATTAN,10003,40.714165,-74.006320,1,...,Unspecified,sedan,Bike,CHAMBERS STREET,33.94,10000.0,61,9.17,Clear,1
1,4268322,"January 2, 2020",16:00:00,Thursday,January,MANHATTAN,10012,40.725643,-73.992070,1,...,Unspecified,sedan,Bike,BOWERY,42.03,10000.0,54,9.17,Clear,1
2,4268207,"January 2, 2020",17:00:00,Thursday,January,BROOKLYN,11221,40.693874,-73.917770,1,...,Unspecified,station wagon/sport utility vehicle,Bike,CENTRAL AVENUE,44.60,10000.0,48,12.75,Rain,0
3,4268408,"January 3, 2020",8:00:00,Friday,January,BRONX,10456,40.820747,-73.906006,1,...,Unspecified,e-bike,Taxi,JACKSON AVENUE,46.78,10000.0,60,8.05,Rain,0
4,4269187,"January 3, 2020",11:00:00,Friday,January,BROOKLYN,11205,40.691017,-73.954475,1,...,Unsafe Speed,sedan,E-Bike,DE KALB AVENUE,44.92,10000.0,74,8.05,Rain,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9188,4571618,"October 9, 2022",1:00:00,Sunday,October,QUEENS,11375,40.734295,-73.850310,1,...,Unspecified,taxi,Bike,108 STREET,52.79,10000.0,53,8.05,Clear,0
9189,4571590,"October 9, 2022",6:00:00,Sunday,October,BROOKLYN,11201,40.689840,-73.978630,1,...,Unspecified,bike,Station Wagon/Sport Utility Vehicle,DE KALB AVENUE,48.74,10000.0,66,8.01,Clear,1
9190,4571393,"October 9, 2022",11:00:00,Sunday,October,BROOKLYN,11201,40.702610,-73.992516,1,...,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,bike,Bike,FRONT STREET,44.96,10000.0,74,4.61,Clear,1
9191,4571698,"October 9, 2022",12:00:00,Sunday,October,MANHATTAN,10002,40.715080,-73.992500,1,...,Unspecified,bike,Station Wagon/Sport Utility Vehicle,CANAL STREET,46.53,10000.0,72,5.75,Clear,1


In [176]:
ML.columns.tolist()

['COLLISION_ID',
 'DATES',
 'TIME',
 'DAY_OF_WEEK',
 'MONTH',
 'BOROUGH',
 'ZIP CODE',
 'LATITUDE',
 'LONGITUDE',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'CONTRIBUTING FACTOR VEHICLE 1',
 'CONTRIBUTING FACTOR VEHICLE 2',
 'VEHICLE TYPE CODE 1',
 'VEHICLE TYPE CODE 2',
 'STREET',
 'TEMP',
 'VISIBILITY',
 'HUMIDITY',
 'WIND_SPEED',
 'WEATHER_MAIN',
 'BIKE_LANE']

In [184]:
ML.groupby(["CONTRIBUTING FACTOR VEHICLE 2"]).count()

,COLLISION_ID,DATES,TIME,DAY_OF_WEEK,MONTH,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN,BIKE_LANE
CONTRIBUTING FACTOR VEHICLE 2,,,,,,,,,,,,,,,,,,,,,
Aggressive Driving/Road Rage,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
Alcohol Involvement,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,15,16,16,16,16
Backing Unsafely,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Brakes Defective,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Cell Phone (hand-Held),3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Driver Inattention/Distraction,1013,1013,1013,1013,1013,1013,1013,1013,1013,1013,...,1013,1013,1013,1013,1013,1012,1013,1013,1013,1013
Driver Inexperience,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
Driverless/Runaway Vehicle,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Drugs (illegal),1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [185]:
ML.groupby(["CONTRIBUTING FACTOR VEHICLE 1"]).count()

,COLLISION_ID,DATES,TIME,DAY_OF_WEEK,MONTH,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN,BIKE_LANE
CONTRIBUTING FACTOR VEHICLE 1,,,,,,,,,,,,,,,,,,,,,
Accelerator Defective,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Aggressive Driving/Road Rage,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
Alcohol Involvement,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
Backing Unsafely,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
Brakes Defective,34,34,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
Cell Phone (hand-Held),3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Driver Inattention/Distraction,2922,2922,2922,2922,2922,2921,2922,2922,2922,2922,...,2922,2922,2922,2922,2922,2914,2922,2922,2922,2922
Driver Inexperience,115,115,115,115,115,115,115,115,115,115,...,115,115,115,115,115,115,115,115,115,115
Driverless/Runaway Vehicle,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [178]:
# getting columns contains strings
df_str_columns=ML.select_dtypes(include=['object'])
for col in df_str_columns.columns:
    print(col)

DATES
TIME
DAY_OF_WEEK
MONTH
BOROUGH
CONTRIBUTING FACTOR VEHICLE 1
CONTRIBUTING FACTOR VEHICLE 2
VEHICLE TYPE CODE 1
VEHICLE TYPE CODE 2
STREET
WEATHER_MAIN


In [179]:
ML["TIME"] =pd.to_datetime(ML["TIME"])

In [180]:
ML["TIME"]=ML["TIME"].astype(str)

In [181]:
ML["TIME"]= ML["TIME"].map(lambda x: x.lstrip('2022-10-25'). rstrip(' ')).map(lambda x: x.lstrip('2022-10-25'). rstrip(' '))

In [182]:
ML["DATES"] =pd.to_datetime(ML["DATES"])

In [183]:
ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9193 entries, 0 to 9192
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   COLLISION_ID                   9193 non-null   int64         
 1   DATES                          9193 non-null   datetime64[ns]
 2   TIME                           9193 non-null   object        
 3   DAY_OF_WEEK                    9193 non-null   object        
 4   MONTH                          9193 non-null   object        
 5   BOROUGH                        9192 non-null   object        
 6   ZIP CODE                       9193 non-null   int64         
 7   LATITUDE                       9193 non-null   float64       
 8   LONGITUDE                      9193 non-null   float64       
 9   NUMBER OF PERSONS INJURED      9193 non-null   int64         
 10  NUMBER OF PERSONS KILLED       9193 non-null   int64         
 11  NUMBER OF PEDESTR

In [142]:
# getting columns contains strings
df_str_columns=ML.select_dtypes(include=['object'])
for col in df_str_columns.columns:
    print(col)

TIME
DAY_OF_WEEK
MONTH
BOROUGH
CONTRIBUTING FACTOR VEHICLE 1
CONTRIBUTING FACTOR VEHICLE 2
VEHICLE TYPE CODE 1
VEHICLE TYPE CODE 2
STREET
WEATHER_MAIN


In [186]:
# Create our features
X = pd.get_dummies(ML, columns= ["TIME", "DAY_OF_WEEK","BOROUGH", 
                                "CONTRIBUTING FACTOR VEHICLE 1", "CONTRIBUTING FACTOR VEHICLE 2", "VEHICLE TYPE CODE 1", "VEHICLE TYPE CODE 2", 
                            "WEATHER_MAIN"]).drop(["COLLISION_ID","MONTH", "LATITUDE","LONGITUDE", "STREET", "DATES", "CONTRIBUTING FACTOR VEHICLE 2" "BIKE_LANE"],axis=1)

# Create our target
y = ML['BIKE_LANE']
X.head()

KeyError: "['CONTRIBUTING FACTOR VEHICLE 2BIKE_LANE'] not found in axis"

In [165]:
X.describe()

,ZIP CODE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,TEMP,...,WEATHER_MAIN_Clear,WEATHER_MAIN_Clouds,WEATHER_MAIN_Drizzle,WEATHER_MAIN_Fog,WEATHER_MAIN_Haze,WEATHER_MAIN_Mist,WEATHER_MAIN_Rain,WEATHER_MAIN_Smoke,WEATHER_MAIN_Snow,WEATHER_MAIN_Thunderstorm
count,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.0,9193.000000,...,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000,9193.000000
mean,10806.107691,1.041771,0.000218,0.001740,0.000109,1.019363,0.000109,0.017622,0.0,64.616714,...,0.498423,0.294681,0.000870,0.000979,0.010116,0.037529,0.142391,0.004569,0.005113,0.005330
std,558.595380,0.242368,0.014749,0.055164,0.010430,0.142461,0.010430,0.170478,0.0,16.271678,...,0.500025,0.455924,0.029488,0.031275,0.100076,0.190063,0.349470,0.067441,0.071323,0.072817
min,10000.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,14.220000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10036.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,52.470000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,11205.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,67.510000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,11228.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,77.310000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,11694.000000,7.000000,1.000000,3.000000,1.000000,3.000000,1.000000,5.000000,0.0,97.270000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [166]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9193 entries, 0 to 9192
Columns: 224 entries, ZIP CODE to WEATHER_MAIN_Thunderstorm
dtypes: float64(3), int64(10), uint8(211)
memory usage: 2.8 MB


In [167]:
np.where(X.values >= np.finfo(np.float64).max)

(array([], dtype=int64), array([], dtype=int64))

In [168]:
# Check the balance of our target values
y.value_counts()

0    5998
1    3195
Name: BIKE_LANE, dtype: int64

In [169]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [170]:
# Check balances
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 4533, 1: 2361})
Counter({0: 1465, 1: 834})


In [171]:
#!pip install imbalanced-learn==0.9.0

In [172]:
#!pip install scikit-learn==1.0.1

In [173]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
BalancedRFC_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
BalancedRFC_model= BalancedRFC_model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [155]:
np.isnan(X)
np.isnan(X.values.any())

False

In [157]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
BalancedRFC_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
BalancedRFC_model= BalancedRFC_model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [83]:
np.isnan(x.values.any())

(array([], dtype=int64), array([], dtype=int64))